<a href="https://colab.research.google.com/github/malborroni/RECMojion/blob/master/Architecture_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerie

In [1]:
! sudo pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-oehw2rqx
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-oehw2rqx
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=fc91d9ad64d0316ea5595560b1db0faa437fb34f1500fbd15113ca5ef68463cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-yb1xxe87/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [2]:
# Example of face detection with a vggface2 model
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import random as rnd

import seaborn as sns
from matplotlib import style
style.use('seaborn-pastel')

import keras
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.models import Model, Sequential, load_model
from keras.utils import to_categorical
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input, decode_predictions
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold

import random
import pickle

Using TensorFlow backend.


In [0]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

# Funzioni

In [0]:
def graph(history):
  # funzione che prende in input il modello fittato e fa un plot della loss in validation e train
  epochs=len(history.history['loss'])
  x_plot = list(range(1,epochs+1))
  plt.figure()
  
  plt.subplot(1, 2, 1)
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.plot(x_plot, history.history['loss'])
  plt.plot(x_plot, history.history['val_loss'])
  plt.legend(['Training', 'Validation'])

  
  plt.subplot(1, 2, 2)
  plt.xlabel('Epochs')
  plt.ylabel('Acc')
  plt.plot(x_plot, history.history['acc'])
  plt.plot(x_plot, history.history['val_acc'])
  plt.legend(['Training', 'Validation'])
  
  plt.tight_layout()
  plt.show()


#Dataset

Il dataset si trova in un file chiamato data_prep.npz nel drive 




In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = np.load('/content/drive/My Drive/AML/data_prep.npz')


# training set
x_train = data['x_train_fe']
y_train = data['y_train']

# validation set
x_val = data['x_val_fe']
y_val = data['y_val']

# test set
x_test = data['x_test_fe']
y_test = data['y_test']

In [0]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train = to_categorical(y_train, num_classes = 7)
y_test = le.fit_transform(y_test)
y_test = to_categorical(y_test, num_classes = 7)
y_val = le.fit_transform(y_val)
y_val = to_categorical(y_val, num_classes = 7)

In [33]:
x_train.shape[1:]

(14, 14, 1024)

# Modello

In [0]:
from keras import regularizers
def create_model(C=1,D=1):
  # Create model
  model = Sequential()
  # C1
  model.add(Conv2D(32, kernel_size=1, activation='relu', input_shape=(14, 14, 1024)))
  model.add(Conv2D(128, kernel_size=3, activation='relu'))
  model.add(Conv2D(32, kernel_size=1, activation='relu'))
  
  if(C>1):
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, kernel_size=1, activation='relu'))
  if(C>2):
    model.add(Conv2D(32, kernel_size=3, activation='relu'))
    model.add(Conv2D(32, kernel_size=1, activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  # D
  model.add(Dense(112, activation='relu',kernel_regularizer=regularizers.l2(0.01))) #210
  model.add(Dropout(0.5))
  if(D>1):
    model.add(Dense(56, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.3))
  if(D>2):
    model.add(Dense(21, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.3))
  
  model.add(Dense(7, activation='softmax'))

  # Compile model
  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['acc'])
  return model

In [69]:
model=create_model(C=3,D=3)
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_141 (Conv2D)          (None, 14, 14, 32)        32800     
_________________________________________________________________
conv2d_142 (Conv2D)          (None, 12, 12, 128)       36992     
_________________________________________________________________
conv2d_143 (Conv2D)          (None, 12, 12, 32)        4128      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_144 (Conv2D)          (None, 6, 6, 32)          1056      
_________________________________________________________________
conv2d_145 (Conv2D)          (None, 4, 4, 32)          9248      
_________________________________________________________________
conv2d_146 (Conv2D)          (None, 4, 4, 32)        